In [2]:
import org.springframework.ai.chat.messages.UserMessage
import org.springframework.ai.chat.model.ChatModel
import org.springframework.ai.chat.model.ChatResponse
import org.springframework.ai.chat.prompt.Prompt
import org.springframework.ai.ollama.OllamaChatModel
import org.springframework.ai.ollama.api.OllamaApi
import org.springframework.ai.ollama.api.OllamaModel
import org.springframework.ai.ollama.api.OllamaOptions

fun indexGraph(chatModel: ChatModel, prompt: Prompt): ChatResponse {
    return chatModel.call(prompt)
}

val tuple_delimiter = "||"
val record_delimiter = ";"
val completion_delimiter = "---COMPLETED---"
val entity_types = listOf(
    "person",
    "mission",
    "organization",
    "location",
    "incident",
    "event",
    "memory",
    "history"
).joinToString { "," }
val input_text = """
Harry James Potter (b. 31 July 1980) was an English half-blood wizard, and one of the most famous wizards of modern times. He was the only child and son of James and Lily Potter , both members of the original Order of the Phoenix. Harry's birth was overshadowed by a prophecy, naming either himself or Neville Longbottom as the one with the power to vanquish Lord Voldemort. After half of the prophecy was reported to Voldemort, courtesy of Severus Snape, Harry was chosen as the target due to his many similarities with the Dark Lord. In turn, this caused the Potter family to go into hiding. Voldemort made his first vain attempt to circumvent the prophecy when Harry was a year and three months old. During this attempt, he murdered Harry's parents as they tried to protect him, but this unsuccessful attempt to kill Harry led to Voldemort's first downfall. This downfall marked the end of the First Wizarding War, and to Harry henceforth being known as the \"Boy Who Lived\", as he was the only known survivor of the Killing Curse. \nOne consequence of Lily's loving sacrifice was that her orphaned son had to be raised by her only remaining blood relative, his Muggle aunt, Petunia Dursley. While in her care he would be protected from Lord Voldemort, due to the Bond of Blood charm Albus Dumbledore placed upon him. This powerful charm would protect him until he became of age, or no longer called his aunt's house home. Due to Petunia's resentment of her sister and her magic gifts, Harry grew up abused and neglected.\nOn his eleventh birthday, Harry learned that he was a wizard, from Rubeus Hagrid. He began attending Hogwarts School of Witchcraft and Wizardry in 1991. The Sorting Hat was initially going to Sort Harry into Slytherin House, but Harry pleaded 'not Slytherin' and the Hat heeded this plea, instead sorting the young wizard into Gryffindor House. At school, Harry became best friends with Ron Weasley and Hermione Granger. He later became the youngest Quidditch Seeker in over a century and eventually the captain of the Gryffindor House Quidditch Team in his sixth year, winning two Quidditch Cups. He became even better known in his early years for protecting the Philosopher's Stone from Voldemort, saving Ron's sister Ginny Weasley, solving the mystery of the Chamber of Secrets, slaying Salazar Slytherin's basilisk, and learning how to conjure a corporeal stag Patronus at the age of thirteen. In his fourth year, Harry won the Triwizard Tournament, although the competition ended with the tragic death of Cedric Diggory and the return of Lord Voldemort. During the next school year, Harry reluctantly taught and led Dumbledore's Army. He also fought in the Battle of the Department of Mysteries, during which he lost his godfather, Sirius Black.\nHarry played a significant role in many other battles of the Second Wizarding War. He, Ron, and Hermione hunted down and destroyed Voldemort's Horcruxes. During the Battle of Hogwarts, Harry personally saw the deaths of Severus Snape and Fred Weasley while learning that Remus Lupin, Nymphadora Tonks, Colin Creevey, and many others had fallen in battle as well. He encountered Voldemort and nearly sacrificed himself, knowing that his doing so was the only way to destroy the fragment of Voldemort's soul inside of him. In limbo, after Voldemort cursed him in the forest clearing, Albus Dumbledore gave Harry advice and background information on the Dark Lord. When told he could choose to live or to \"move on\", Harry chose to live. After he awoke, Harry confronted Voldemort and defeated him once and for all. \nHarry was also noted for being the only known Master of Death, having united the three Deathly Hallows at the mere age of seventeen.\nAfter the war, Harry became an Auror and helped reform and revolutionise the Ministry of Magic. At some point, he married Ginny Weasley, with whom he had three children: James Sirius (named after his father and godfather), Albus Severus (named after Albus Dumbledore and Severus Snape), and Lily Luna (named after his mother and Luna Lovegood). He was also named the godfather of Edward \"Teddy\" Remus Lupin. In 2007 Harry was promoted to Head of the Auror Office at the age of 26, and would occasionally deliver Defence Against the Dark Arts lectures at Hogwarts. He later went on to be Head of the Department of Magical Law Enforcement by the summer of 2020.\n
"""

val messageOne = """
        Analyze the text in tag <text>. Think step by step to generate a knowledge graph to model entity and relationships. Put the observation in tag <thinking>.
        Then use the following steps to deduce all relations among the entities. Output as more as possible.

        Extract the following entity information in tag <entities>:
        - {entity_name}: Name of the entity, capitalized
        - {entity_type}: Type of the entity, use nounce and uppercase
        Format each entity as ({entity_name}${tuple_delimiter}{entity_type}).
        When finished, add ${completion_delimiter} before closing the tag.

        ### Example
        <text>Peter's best friend is Mary. They are neighbour and grow up together since age 6 in Hong Kong.</text>
        <thinking>
        Observations: There are two person and one location.
        Deduced entity types: PERSON, LOCATION
        </thinking>
        <entities>
        (Peter${tuple_delimiter}PERSON)${record_delimiter}
        (Mary${tuple_delimiter}PERSON)${record_delimiter}
        (Hong Kong${tuple_delimiter}LOCATION)${record_delimiter}
        ${completion_delimiter}
        </entities>
        #############################

        ### Real data
        <text>${input_text}</text>
        """.trimIndent()
val messageTwo = """
        Analyze <entities> relations base on <text> and each entity must have at least one relations, extract the following information in tag <relations>:
        - {source_entity}: name of the source entity, as identified in step 1
        - {target_entity}: name of the target entity, as identified in step 1
        - {relation_type}: describe source and target entity relationship, use verb, adjective and uppercase
        - {relation_score}: a numeric score indicating strength of the relationship between the source entity and target entity
         Format each relationship as ({source_entity}${tuple_delimiter}{relation_type}${tuple_delimiter}{target_entity}${tuple_delimiter}{relation_score}).
         When finished, add ${completion_delimiter} before closing the tag.
         ### Example
         <relations>
         (Peter${tuple_delimiter}IS_BEST_FRIEND_OF${tuple_delimiter}Mary${tuple_delimiter}9)${record_delimiter}
         (Peter${tuple_delimiter}IS_NEIGHBOUR_OF${tuple_delimiter}Mary${tuple_delimiter}9)${record_delimiter}
         (Peter${tuple_delimiter}LIVES_IN${tuple_delimiter}Hong Kong${tuple_delimiter}4)${record_delimiter}
         (Mary${tuple_delimiter}LIVES_IN${tuple_delimiter}Hong Kong${tuple_delimiter}4)${record_delimiter}
         ${completion_delimiter}
         </relations>
        #############################
""".trimIndent()

val gemma2_9B = OllamaChatModel.builder()
    .withOllamaApi(OllamaApi("http://localhost:11434"))
    .withDefaultOptions(OllamaOptions()
        .withModel("gemma2:9b")
        .withNumCtx(8192)
        .withTopP(0.5)
        .withTopK(100)
        .withTemperature(0.2))
    .build()
val messageThree = """
        Contrust a mermaid class diagram in tag <graph-schema> to represent the graph schema for entity_type and relation_type
        ### Example
        <graph-schema>
        ```mermaid
        classDiagram
            PERSON -- PERSON: IS_BEST_FRIEND_OF, IS_NEIGHBOUR_OF
            PERSON -- LOCATION: LIVES_IN
        ```
        ${completion_delimiter}
        </graph-schema>
        #############################
"""
val chatResponse = indexGraph(
    gemma2_9B,
    Prompt(
        listOf(UserMessage(messageOne))
    )
)


print(chatResponse.result)
print(chatResponse.metadata.usage)

val chatResponse2 = indexGraph(
    gemma2_9B,
    Prompt(
        listOf(
            UserMessage(messageOne),
            chatResponse.result.output,
            UserMessage(messageTwo)
        )
    )
)

print(chatResponse2.result)
print(chatResponse2.metadata.usage)

val chatResponse3 = indexGraph(
    gemma2_9B,
    Prompt(
        listOf(
            UserMessage(messageOne),
            chatResponse.result.output,
            UserMessage(messageTwo),
            chatResponse2.result.output,
            UserMessage(messageThree),
        )
    )
)

print(chatResponse3.result)
print(chatResponse3.metadata.usage)


Generation[assistantMessage=AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent=<thinking>
Observations: The text describes Harry Potter's life, mentioning his family, friends, enemies, school, and accomplishments. There are many named individuals, organizations, magical objects, and events.

Deduced entity types: PERSON, ORGANIZATION, LOCATION, EVENT, MAGIC_OBJECT, CONCEPT


</thinking>
<entities>
(Harry James Potter||PERSON);
(James Potter||PERSON);
(Lily Potter||PERSON);
(Neville Longbottom||PERSON);
(Lord Voldemort||PERSON);
(Severus Snape||PERSON);
(Albus Dumbledore||PERSON);
(Petunia Dursley||PERSON);
(Rubeus Hagrid||PERSON);
(Ron Weasley||PERSON);
(Hermione Granger||PERSON);
(Ginny Weasley||PERSON);
(Cedric Diggory||PERSON);
(Sirius Black||PERSON);
(Remus Lupin||PERSON);
(Nymphadora Tonks||PERSON);
(Colin Creevey||PERSON);
(Edward \"Teddy\" Remus Lupin||PERSON);
(Order of the Phoenix||ORGANIZATION);
(Hogwarts School of Witchcraft and Wizardry||ORGANIZATION);
(Slyt

# Extraction

In [16]:
val content = """
${chatResponse.results[0].output.content}
${chatResponse2.results[0].output.content}
${chatResponse3.results[0].output.content}
""".trimIndent()

val entitiesTagRe = """<entities>([\s\S]*?)</entities>""".toRegex()
val relationsTagRe = """<relations>([\s\S]*?)</relations>""".toRegex()
val schemaTagRe = """<graph-schema>([\s\S]*?)</graph-schema>""".toRegex()

outputTagRe.findAll(content).map { it.groupValues[1].trim() }.take(1)

val entities = entitiesTagRe.find(content)?.groupValues?.get(1)?.trim() ?: ""

val relations = relationsTagRe.find(content)?.groupValues?.get(1)?.trim() ?: ""
val graphSchema = schemaTagRe.find(content)?.groupValues?.get(1)?.trim() ?: ""

println(entities)
println(relations)
println(graphSchema)
// more post-processing can be done

(Harry James Potter||PERSON);
(James Potter||PERSON);
(Lily Potter||PERSON);
(Neville Longbottom||PERSON);
(Lord Voldemort||PERSON);
(Severus Snape||PERSON);
(Albus Dumbledore||PERSON);
(Petunia Dursley||PERSON);
(Rubeus Hagrid||PERSON);
(Ron Weasley||PERSON);
(Hermione Granger||PERSON);
(Ginny Weasley||PERSON);
(Cedric Diggory||PERSON);
(Sirius Black||PERSON);
(Remus Lupin||PERSON);
(Nymphadora Tonks||PERSON);
(Colin Creevey||PERSON);
(Edward \"Teddy\" Remus Lupin||PERSON);
(Order of the Phoenix||ORGANIZATION);
(Hogwarts School of Witchcraft and Wizardry||ORGANIZATION);
(Slytherin House||ORGANIZATION);
(Gryffindor House||ORGANIZATION);
(Ministry of Magic||ORGANIZATION);
(Department of Mysteries||LOCATION);
(First Wizarding War||EVENT);
(Second Wizarding War||EVENT);
(Battle of the Department of Mysteries||EVENT);
(Battle of Hogwarts||EVENT);
(Triwizard Tournament||EVENT);
(Killing Curse||MAGIC_OBJECT);
(Bond of Blood charm||MAGIC_OBJECT);
(Sorting Hat||MAGIC_OBJECT);
(Philosopher's St